In [136]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-2023-dataset/Cleaned_data_for_model.csv
/kaggle/input/house-prices-2023-dataset/For_EDA_dataset.csv
/kaggle/input/house-prices-2023-dataset/House_Price_dataset.csv


In [115]:
df=pd.read_csv("/kaggle/input/house-prices-2023-dataset/Cleaned_data_for_model.csv")
df.drop("Unnamed: 0",axis=1,inplace=True)
df

,property_type,price,location,city,baths,purpose,bedrooms,Area_in_Marla
0,Flat,10000000,G-10,Islamabad,2,For Sale,2,4.0
1,Flat,6900000,E-11,Islamabad,3,For Sale,3,5.6
2,House,16500000,G-15,Islamabad,6,For Sale,5,8.0
3,House,43500000,Bani Gala,Islamabad,4,For Sale,4,40.0
4,House,7000000,DHA Defence,Islamabad,3,For Sale,3,8.0
...,...,...,...,...,...,...,...,...
99494,Flat,7500000,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99495,House,8800000,Bahria Town Karachi,Karachi,4,For Sale,3,8.0
99496,House,14000000,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99497,House,14000000,Bahria Town Karachi,Karachi,4,For Sale,4,14.0


In [116]:
df.describe()

,price,baths,bedrooms,Area_in_Marla
count,9.949900e+04,99499.000000,99499.000000,99499.000000
mean,1.037592e+07,3.530106,3.350737,8.757479
std,1.066000e+07,1.474217,1.255913,7.466471
min,1.550000e+04,1.000000,0.000000,0.000000
25%,1.500000e+05,2.000000,2.000000,4.800000
50%,7.500000e+06,3.000000,3.000000,6.700000
75%,1.550000e+07,5.000000,4.000000,10.000000
max,4.490000e+07,7.000000,6.000000,194.000000


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99499 entries, 0 to 99498
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_type  99499 non-null  object 
 1   price          99499 non-null  int64  
 2   location       99499 non-null  object 
 3   city           99499 non-null  object 
 4   baths          99499 non-null  int64  
 5   purpose        99499 non-null  object 
 6   bedrooms       99499 non-null  int64  
 7   Area_in_Marla  99499 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 6.1+ MB


In [118]:
df["property_type"].unique()

array(['Flat', 'House', 'Penthouse', 'Lower Portion', 'Upper Portion',
       'Room', 'Farm House'], dtype=object)

In [119]:
df["city"].unique()

array(['Islamabad', 'Lahore', 'Faisalabad', 'Rawalpindi', 'Karachi'],
      dtype=object)

In [120]:
df["baths"].unique()

array([2, 3, 6, 4, 5, 1, 7])

In [121]:
df["purpose"].unique()

array(['For Sale', 'For Rent'], dtype=object)

In [122]:
df["bedrooms"].unique()

array([2, 3, 5, 4, 1, 6, 0])

In [123]:
df[df["purpose"]=="For Sale"]["price"].mean()

14517334.023989739

In [124]:
df[df["purpose"]=="For Rent"]["price"].mean()

85189.02108433735

In [125]:
df=df[df["price"]>0]
df

,property_type,price,location,city,baths,purpose,bedrooms,Area_in_Marla
0,Flat,10000000,G-10,Islamabad,2,For Sale,2,4.0
1,Flat,6900000,E-11,Islamabad,3,For Sale,3,5.6
2,House,16500000,G-15,Islamabad,6,For Sale,5,8.0
3,House,43500000,Bani Gala,Islamabad,4,For Sale,4,40.0
4,House,7000000,DHA Defence,Islamabad,3,For Sale,3,8.0
...,...,...,...,...,...,...,...,...
99494,Flat,7500000,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99495,House,8800000,Bahria Town Karachi,Karachi,4,For Sale,3,8.0
99496,House,14000000,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99497,House,14000000,Bahria Town Karachi,Karachi,4,For Sale,4,14.0


In [126]:
df1=df[df["purpose"]=="For Sale"]
df1

,property_type,price,location,city,baths,purpose,bedrooms,Area_in_Marla
0,Flat,10000000,G-10,Islamabad,2,For Sale,2,4.0
1,Flat,6900000,E-11,Islamabad,3,For Sale,3,5.6
2,House,16500000,G-15,Islamabad,6,For Sale,5,8.0
3,House,43500000,Bani Gala,Islamabad,4,For Sale,4,40.0
4,House,7000000,DHA Defence,Islamabad,3,For Sale,3,8.0
...,...,...,...,...,...,...,...,...
99494,Flat,7500000,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99495,House,8800000,Bahria Town Karachi,Karachi,4,For Sale,3,8.0
99496,House,14000000,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99497,House,14000000,Bahria Town Karachi,Karachi,4,For Sale,4,14.0


In [127]:
df2=df[df["purpose"]=="For Rent"]
df2

,property_type,price,location,city,baths,purpose,bedrooms,Area_in_Marla
22,House,110000,DHA Defence,Islamabad,5,For Rent,5,20.0
23,House,600000,F-7,Islamabad,6,For Rent,6,40.0
24,Lower Portion,33000,G-15,Islamabad,2,For Rent,2,8.0
25,House,45000,Pakistan Town,Islamabad,5,For Rent,3,24.0
26,House,175000,F-7,Islamabad,4,For Rent,4,13.3
...,...,...,...,...,...,...,...,...
99439,Upper Portion,115000,I-8,Islamabad,3,For Rent,4,20.0
99440,Lower Portion,45000,PWD Housing Scheme,Islamabad,3,For Rent,3,12.0
99441,House,400000,F-7,Islamabad,6,For Rent,5,26.0
99442,House,100000,E-11,Islamabad,7,For Rent,5,64.0


In [128]:
x=df1.drop("price",axis=1)
y=df1["price"]
x

,property_type,location,city,baths,purpose,bedrooms,Area_in_Marla
0,Flat,G-10,Islamabad,2,For Sale,2,4.0
1,Flat,E-11,Islamabad,3,For Sale,3,5.6
2,House,G-15,Islamabad,6,For Sale,5,8.0
3,House,Bani Gala,Islamabad,4,For Sale,4,40.0
4,House,DHA Defence,Islamabad,3,For Sale,3,8.0
...,...,...,...,...,...,...,...
99494,Flat,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99495,House,Bahria Town Karachi,Karachi,4,For Sale,3,8.0
99496,House,Bahria Town Karachi,Karachi,3,For Sale,3,8.0
99497,House,Bahria Town Karachi,Karachi,4,For Sale,4,14.0


In [129]:
y

0        10000000
1         6900000
2        16500000
3        43500000
4         7000000
           ...   
99494     7500000
99495     8800000
99496    14000000
99497    14000000
99498     9000000
Name: price, Length: 70947, dtype: int64

In [130]:
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,1,2,4])],remainder='passthrough')
x=ct.fit_transform(x)
x

<70947x1348 sparse matrix of type '<class 'numpy.float64'>'
	with 496416 stored elements in Compressed Sparse Row format>

In [131]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [132]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()

model.fit(X_train,Y_train)
linear_model=model.score(X_test,Y_test)
linear_model

0.7030340951844769

In [133]:
from sklearn import tree
dt_model=tree.DecisionTreeRegressor()
dt_model.fit(X_train,Y_train)
dt_score=dt_model.score(X_test,Y_test)
dt_score

0.8425803722276485

In [134]:
from sklearn.ensemble import RandomForestRegressor
random_forest_model=RandomForestRegressor()
random_forest_model.fit(X_train,Y_train)
random_forest_score=random_forest_model.score(X_test,Y_test)
random_forest_score

0.8648905159588973

In [135]:
from sklearn.neighbors import KNeighborsRegressor
knn_model=KNeighborsRegressor()
knn_model.fit(X_train,Y_train)
knn_score=knn_model.score(X_test,Y_test)
knn_score

0.8454208853576665

In [137]:
rf_grid = {"n_estimators": [50, 100, 150, 200],
           "max_depth": [None, 5, 10, 15],
           "min_samples_split": [2, 5, 10, 20],
           "max_features": ['auto', 'sqrt', 'log2'],
           "max_samples": [0.5, 0.7, 0.9]}

In [138]:
rf_random = RandomizedSearchCV(estimator = RandomForestRegressor(), 
                               param_distributions = rf_grid,
                               n_iter = 2,
                               cv = 5,
                               verbose=2,
                               random_state=42,
                               n_jobs = -1)
rf_random.fit(X_train,Y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=2, n_jobs=-1,
                   param_distributions={'max_depth': [None, 5, 10, 15],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'max_samples': [0.5, 0.7, 0.9],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [50, 100, 150, 200]},
                   random_state=42, verbose=2)

In [139]:
rf_random.best_params_


{'n_estimators': 150,
 'min_samples_split': 5,
 'max_samples': 0.5,
 'max_features': 'log2',
 'max_depth': None}

In [140]:
rf_random.score(X_train,Y_train)

0.9097356927511183